<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

## Getting Started

Let's start importing libraries and creating useful variables 

In [ ]:
%load_ext autotime

In [ ]:
from confluent_kafka import avro
from confluent_kafka.avro import AvroProducer
import time
import qcutils

servers=qcutils.read_config_value("kafka.server") + ":" + str(qcutils.read_config_value("kafka.port"))

sr_url=qcutils.read_config_value("kafka.schema_registry.url")

topic = ''

qcutils.create_kafka_topic(topic)

# write a kafka avro async producer

Define avro schemas for keys and value to describe the throw of the dice.

### Key
The key has a single field to contain arbitrary `Name`

### Value
A throw of a dice is described by an int `Number`.

The producer **MUST** be asynchronous and the results of the production must be checked.

1) define the key schema

In [ ]:
key_schema_str = """
{
  "namespace": "lab.avro",
  "type": "record",
  "name": "DiceTh",
  "fields": [
    {
      "name": "name",
      "type": "string"
    }
  ]
}
"""

2) define the value schema

In [ ]:
value_schema_str = """
{
  "namespace": "lab.avro",
  "type": "record",
  "name": "DiceValue",
  "fields": [
    {
      "name": "number",
      "type": "int"
    }
  ]
}
"""

3) create a topic `<surname>-avro-dice` and send messages

In [ ]:
key_schema = avro.loads(key_schema_str)
value_schema = avro.loads(value_schema_str)

In [ ]:
def delivery_report(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

producerconf = {
        'bootstrap.servers': servers,
        'schema.registry.url': sr_url
    }
        
ap = AvroProducer(producerconf, default_key_schema=key_schema, default_value_schema=value_schema, on_delivery=delivery_report)

key = {"name": "one"}
value = {"number": 1}
ap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema, on_delivery=delivery_report)
print(value)
ap.poll(1)

key = {"name": "six"}
value = {"number": 6}
ap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema, on_delivery=delivery_report)
print(value)
ap.poll(1)

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.